In [ ]:
# ML - project 2 Model traing

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from pathlib import Path
import matplotlib.pyplot as plt
import matplotlib
import sklearn
import re
from category_encoders import TargetEncoder


# different Model imports
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import GradientBoostingRegressor

# import metrics for evaluating the model
from sklearn.metrics import mean_squared_error

# import for splitting the data so we get 75 % train-data and 25 %  test-data'
from sklearn.model_selection import train_test_split 

# Trun categories into number
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer

# used to reduced the datas compelixty
#from sklearn.decomposition import PCA

# Setps for machine work-flow
# 1.Read the Data: Load the data from a CSV or other data source
# 2.Preprocess the Data: Clean the data, handle missing values, encode categorical variables, scale numerical features, etc.
# 3.Split the Data: Use train_test_split to divide the data into training and testing sets.
# 4.Train the Model(s): Fit your chosen machine learning model(s) on the training data X_train and y_train.
# 5.Evaluate the Model(s): Use the testing data X_test and y_test to evaluate the performance of your model(s).

file_path = r'C:\Users\Eric\OneDrive\Skrivebord\Dat158 - ML project 2\train.csv'
# Read the CSV file into a DataFrame
train_df = pd.read_csv(file_path)

# Preprocess the Data: Clean the data
# removed id and price form the data frame. Make the columns with string values to integers.

#print(train_df['brand'].value_counts())
#print(train_df['ext_col'].value_counts())
#print(train_df['transmission'].value_counts())
#print(train_df['model'].value_counts())
#print(train_df['fuel_type'].value_counts())


# Split the Data: 75 - train og 25 test
# target_variable is the column that holdes the price in the data-frame. id_column holdes the id values.
# make these as variables, so we can easy swap them out later with something else if needed or use them to edit there format.
target_variable = 'price'
id_column = 'id'
brand_column = 'brand'
modelName_column = 'model'
fuelType_colum = 'fuel_type'
engine_column = 'engine'
transmission_column = 'transmission'
extColor_column = 'ext_col'
intColor_column = 'int_col'
accident_column = 'accident'
cleanTile_column = 'clean_title'


# Separate features and target in the training data
X = train_df.drop(columns=[target_variable, id_column,intColor_column,cleanTile_column])
y = train_df[target_variable]

print("Test size start (rows, columns):")
print(X.shape)

# Extract  horsepower for egnine column using regex
def extract_engine_info(engine_str):
    if pd.isnull(engine_str):
        return np.nan
    hp_match = re.search(r'(\d+\.?\d*)HP', engine_str)
    hp = float(hp_match.group(1)) if hp_match else np.nan
    return hp

# Applying the function to the engine column
X[engine_column] = X[engine_column].apply(extract_engine_info)


# Fill NaNs in the engine column with the mean value of the column
X['engine'].fillna(X['engine'].mean(), inplace=True)


print("Test size after engine info (rows, columns):")
print(X.shape)

# Define the function for checking the accident information
def accident_info(accident_str):
    # Check if the string is None or NaN
    if pd.isnull(accident_str):
        return 0  # Assuming NaN means no report, hence 0
    return 0 if accident_str.strip().lower() == 'none reported' else 1

# Applying the function to the accident_column
X[accident_column] = X[accident_column].apply(accident_info)


print("Test size after accident info (rows, columns):")
print(X.shape)


# set a threshold for the minimum number of occurenences for a brand to be a seperate category
threshold_brand = 500

# Count occurrences for each brand
brand_counts = X[brand_column].value_counts()

# Identify brands that are common
commond_brand = brand_counts[brand_counts >= threshold_brand].index

# teste kun for en bil type audi: X.where(X[brand_column].isin('audi')) ???????????????
# Group the less frequent brans into 'other' using .where()
X[brand_column] = X[brand_column].where(X[brand_column].isin(commond_brand), 'other')

# One-hot encode the 'brand' column so we dont use the String-values
X = pd.get_dummies(X, columns=[brand_column])


# set a threshold for the minimum number of occurenences for a color to be a seperate category
threshold_extcolor = 1000

# Count occurrences for each brand
extColor_counts = X[extColor_column].value_counts()

# Identify color that are common
commond_color = extColor_counts[extColor_counts >= threshold_extcolor].index

# Group the less frequent brans into 'other' using .where()
X[extColor_column] = X[extColor_column].where(X[extColor_column].isin(commond_color), 'other')

# One-hot encode the 'ext_col' column so we dont use the String-values
X = pd.get_dummies(X, columns=[extColor_column])


# set a threshold for the minimum number of occurenences for a transmission to be a seperate category
threshold_transmission = 500

# Count occurrences for each transmission
transmission_counts = X[transmission_column].value_counts()

# Identify transmission that are common
commond_transmission = transmission_counts[transmission_counts >= threshold_extcolor].index

# Group the less frequent transmission into 'other' using .where()
X[transmission_column] = X[transmission_column].where(X[transmission_column].isin(commond_transmission), 'other')

# One-hot encode the 'transmission' column so we dont use the String-values
X = pd.get_dummies(X, columns=[transmission_column])


# set a threshold for the minimum number of occurenences for a model-name to be a seperate category
threshold_modelName = 500

# Count occurrences for each transmission
modelName_counts = X[modelName_column].value_counts()

# Identify transmission that are common
commond_modelName = modelName_counts[modelName_counts >= threshold_extcolor].index

# Group the less frequent transmission into 'other' using .where()
X[modelName_column] = X[modelName_column].where(X[modelName_column].isin(commond_modelName), 'other')

# One-hot encode the 'transmission' column so we dont use the String-values
X = pd.get_dummies(X, columns=[modelName_column])


# set a threshold for the minimum number of occurenences for a fuel-type to be a seperate category
threshold_fuelType = 500

# Count occurrences for each fuel-type
fuelType_counts = X[fuelType_colum].value_counts()

# Identify fuel-type that are common
commond_fuelType = fuelType_counts[fuelType_counts >= threshold_extcolor].index

# Group the less frequent fuel-type into 'other' using .where()
X[fuelType_colum] = X[fuelType_colum].where(X[fuelType_colum].isin(commond_fuelType), 'other')

# One-hot encode the 'fuel_type' column so we dont use the String-values
X = pd.get_dummies(X, columns=[fuelType_colum])



""" # Initialize TargetEncoder
encoder = TargetEncoder(cols=[brand_column])

# Fit and transform the feature dataframe
X_transformed = encoder.fit_transform(X, y)

print("Test size after encoder (rows, columns):")
print(X_transformed.shape)

# Adding the transformed column back to the original dataframe
X[brand_column] = X_transformed[brand_column]

print(X[brand_column].head(10)) """

print("Test size after brand-merge info (rows, columns):")
print(X.shape)

# Specify the categorical features
""" categorical_features = [modelName_column, fuelType_colum, transmission_column,extColor_column] """

# Initialize and configure the transformer
""" one_hot = OneHotEncoder(handle_unknown='ignore')
transformer = ColumnTransformer([("one_hot",
                                 one_hot,
                                 categorical_features)],
                                 remainder="passthrough")

print("Test size before splitt (rows, columns):")
print(X.shape) """

# splitting the data into train and test sets (75% train and 25% test)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)

# Fit the transformer on the training data
""" transformer.fit(X_train) """

# Transform both the training and test data using the fitted transformer
""" X_train_tranformed = transformer.fit_transform(X_train)
X_test_tranformed = transformer.transform(X_test) """


# Create an instance of a model:
ln_reg = LinearRegression()
rf_reg = RandomForestRegressor()
gb_reg = GradientBoostingRegressor()

# Train different models with .fit:
#ln_reg.fit(X_train, y_train)
#rf_reg.fit(X_train, y_train)
gb_reg.fit(X_train, y_train)

# Make a prediction of the test
#y_pred = ln_reg.predict(X_test)
#y_pred = rf_reg.predict(X_test)
y_pred = gb_reg.predict(X_test)

# Evaluate the model
mse = mean_squared_error(y_test, y_pred)
rmse = np.sqrt(mse)
#rmse = root_mean_squared_error(y_test, y_pred)

 
print(f"Root Mean Squared Error (RMSE): {rmse:.4f}")


Test size start (rows, columns):
(188533, 9)
Test size after engine info (rows, columns):
(188533, 9)
Test size after accident info (rows, columns):
(188533, 9)
Test size after brand-merge info (rows, columns):
(188533, 101)
Root Mean Squared Error (RMSE): 72443.9212
